In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
train = pd.read_csv('../input/resolving-citizens-grievances-v1/dataset/train.csv')
test = pd.read_csv('../input/resolving-citizens-grievances-v1/dataset/test.csv')
train.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (16,17,18,19,20,22,23,24,25,26,27,28,45,49,50,51) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (14,15,16,17,18,19,20,22,23,24,25,49,50,51) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,appno,application,country.alpha2,country.name,decisiondate,docname,doctypebranch,ecli,introductiondate,issue.0,...,ccl_article=6,ccl_article=7,ccl_article=8,ccl_article=9,ccl_article=p1,ccl_article=p12,ccl_article=p4,ccl_article=p6,ccl_article=p7,importance
0,2052/08,MS WORD,ru,Russian Federation,NaN,CASE OF KOKOSHKINA v. RUSSIA,CHAMBER,ECLI:CE:ECHR:2009:0528JUD000205208,NaN,NaN,...,0,0,0,0,0,0,0,0,0,4
1,4728/07,MS WORD,tr,Turkey,NaN,CASE OF ÖZCAN v. TURKEY,COMMITTEE,ECLI:CE:ECHR:2018:0710JUD000472807,NaN,NaN,...,0,0,0,0,0,0,0,0,0,4
2,44135/06,MS WORD,si,Slovenia,NaN,CASE OF DANIJEL PEČNIK v. SLOVENIA,CHAMBER,ECLI:CE:ECHR:2012:1018JUD004413506,NaN,NaN,...,1,0,0,0,0,0,0,0,0,4
3,2607/08,MS WORD,ch,Switzerland,NaN,CASE OF PALANCI v. SWITZERLAND,CHAMBER,ECLI:CE:ECHR:2014:0325JUD000260708,NaN,NaN,...,0,0,-1,0,0,0,0,0,0,4
4,27001/06,MS WORD,ru,Russian Federation,NaN,CASE OF AMANAT ILYASOVA AND OTHERS v. RUSSIA,CHAMBER,ECLI:CE:ECHR:2009:1001JUD002700106,NaN,NaN,...,0,0,0,0,0,0,0,0,0,4


In [3]:
train.shape, test.shape

((8878, 328), (4760, 327))

In [4]:
# combining test and train to do feature engineering.
test['importance']=-1
train['label'] = 'train'
test['label'] = 'test'
combined = pd.concat([train,test],axis=0)
combined.shape

(13638, 329)

In [5]:
for col in combined.columns:
    print(col,'unique values-> ',combined[col].nunique(), 'null values--> ', combined[col].isnull().sum())

appno unique values->  13467 null values-->  0
application unique values->  1 null values-->  0
country.alpha2 unique values->  46 null values-->  0
country.name unique values->  46 null values-->  0
decisiondate unique values->  274 null values-->  12603
docname unique values->  13201 null values-->  0
doctypebranch unique values->  3 null values-->  0
ecli unique values->  13638 null values-->  0
introductiondate unique values->  720 null values-->  12602
issue.0 unique values->  3095 null values-->  10162
issue.1 unique values->  1710 null values-->  11780
issue.10 unique values->  29 null values-->  13609
issue.11 unique values->  25 null values-->  13613
issue.12 unique values->  21 null values-->  13617
issue.13 unique values->  16 null values-->  13622
issue.14 unique values->  14 null values-->  13624
issue.15 unique values->  11 null values-->  13627
issue.16 unique values->  7 null values-->  13631
issue.17 unique values->  5 null values-->  13633
issue.18 unique values->  4 

In [6]:
def combine_issues(df):
    issue_columns = [
        'issue.0', 'issue.1', 'issue.2', 'issue.3', 'issue.4', 'issue.5', 'issue.6', 'issue.7', 'issue.8', 
        'issue.9', 'issue.10', 'issue.11', 'issue.12', 'issue.13', 'issue.14', 'issue.15', 'issue.16', 
        'issue.17', 'issue.18', 'issue.19', 'issue.20', 'issue.21', 'issue.22', 'issue.23']
    issue_df = combined[issue_columns]
    issue_df.fillna('',inplace=True)
    issue_df['issues'] = issue_df[issue_columns].apply(lambda x: '. '.join([val for val in x if val != '']), axis=1)
    df.drop(issue_columns, axis=1, inplace=True)
    issue_df.drop(issue_columns, axis=1, inplace=True)
    df = pd.concat([df, issue_df], axis=1)
    return df

In [7]:
def lowercase_texts(df):
    print('converting all text columns in lowercase.',)
    for col in combined.columns:
        if combined[col].dtype=='object':
            combined[col] = combined[col].str.lower()
    return df

In [8]:
def universalize_countries(df):
# converting all the countries to single symbolic numerical value.(eg - Albania, albania, abl, ab -> 1)
    country_dict_A = df[['respondentOrderEng','country.name']].set_index('country.name').T.to_dict('list')
    country_dict_C = df[['respondentOrderEng','respondent.0']].set_index('respondent.0').T.to_dict('list')    
    country_dict = {}
    for d in (country_dict_A, country_dict_C):#, country_dict_C): #, country_dict_D, country_dict_E, country_dict_F): 
        country_dict.update(d)
        
    country_dict = {k: v for k, v in country_dict.items() if pd.notna(k)}
    df['respondent.0'] = df['respondent.0'].apply(lambda x: country_dict[x][0])
    df['respondent.1'] = df['respondent.1'].apply(lambda x: country_dict[x][0] if pd.notnull(x) else x)
    df['respondent.2'] = df['respondent.2'].apply(lambda x: country_dict[x][0] if pd.notnull(x) else x)
    df['respondent.3'] = df['respondent.3'].apply(lambda x: country_dict[x][0] if pd.notnull(x) else x)
    df['respondent.4'] = df['respondent.4'].apply(lambda x: country_dict[x][0] if pd.notnull(x) else x)
    del df['respondentOrderEng']
    return df

In [9]:
def remove_constant_values(df):
#     this function removes redundant constant features.
    print('Removing constant columns -> ',)
    for col in df.columns:
        if df[col].nunique()==1:
            print(col,end=', ' )
            del df[col]
    return df

def remove_unwanted_features(df):
#     these features dont add any valueable signal to the data.
    remove_cols =['parties.0', 'country.alpha2', 'parties.1', 'country.name', 'docname', 'appno', 'ecli', 'kpdate', 'originatingbody_name']
    for col in remove_cols:
        if col in df.columns:
            df.drop(col, axis=1, inplace=True)
    return df

In [10]:
def featurize_columns(df):
#     making new columns.
    df['itemid'] = df['itemid'].apply(lambda x: x[4:7])
    df['sharepointid'] = df['sharepointid'].apply(lambda x: str(x)[:3])
    df['total_respondents'] = 5- df[['respondent.0','respondent.1','respondent.2','respondent.3','respondent.4']].isna().sum(axis=1)

    return df

def featurize_date_columns(df):
    #     making new columns based on dates.
    df['daysbetween_intro_decision'] = (pd.to_datetime(df['decisiondate']) - pd.to_datetime(df['introductiondate'])).dt.days
    df['daysbetween_intro_judgement'] = (pd.to_datetime(df['judgementdate']) - pd.to_datetime(df['introductiondate'])).dt.days
    df['daysbetween_decision_judgement'] = (pd.to_datetime(df['judgementdate']) - pd.to_datetime(df['decisiondate'])).dt.days
    df.drop(['decisiondate','introductiondate','judgementdate'], axis=1, inplace=True)
    return df

In [11]:
le = preprocessing.LabelEncoder()
def encoding(df):
    df['doctypebranch'] = le.fit_transform(df['doctypebranch'])
    df['separateopinion'] = le.fit_transform(df['separateopinion'])
    df['typedescription'] = le.fit_transform(df['typedescription'])
    return df

In [12]:
def fill_missing(df):
    for col in df.columns:
        if col not in ['label', 'issues']:
            df[col].fillna(0,inplace=True)
            df[col] = df[col].astype('int')
    return df

In [13]:
combined.head()

,appno,application,country.alpha2,country.name,decisiondate,docname,doctypebranch,ecli,introductiondate,issue.0,...,ccl_article=7,ccl_article=8,ccl_article=9,ccl_article=p1,ccl_article=p12,ccl_article=p4,ccl_article=p6,ccl_article=p7,importance,label
0,2052/08,MS WORD,ru,Russian Federation,NaN,CASE OF KOKOSHKINA v. RUSSIA,CHAMBER,ECLI:CE:ECHR:2009:0528JUD000205208,NaN,NaN,...,0,0,0,0,0,0,0,0,4,train
1,4728/07,MS WORD,tr,Turkey,NaN,CASE OF ÖZCAN v. TURKEY,COMMITTEE,ECLI:CE:ECHR:2018:0710JUD000472807,NaN,NaN,...,0,0,0,0,0,0,0,0,4,train
2,44135/06,MS WORD,si,Slovenia,NaN,CASE OF DANIJEL PEČNIK v. SLOVENIA,CHAMBER,ECLI:CE:ECHR:2012:1018JUD004413506,NaN,NaN,...,0,0,0,0,0,0,0,0,4,train
3,2607/08,MS WORD,ch,Switzerland,NaN,CASE OF PALANCI v. SWITZERLAND,CHAMBER,ECLI:CE:ECHR:2014:0325JUD000260708,NaN,NaN,...,0,-1,0,0,0,0,0,0,4,train
4,27001/06,MS WORD,ru,Russian Federation,NaN,CASE OF AMANAT ILYASOVA AND OTHERS v. RUSSIA,CHAMBER,ECLI:CE:ECHR:2009:1001JUD002700106,NaN,NaN,...,0,0,0,0,0,0,0,0,4,train


In [14]:
combined = combine_issues(combined)
print('combined shape after combining issues ->', combined.shape)
combined = lowercase_texts(combined)
combined = universalize_countries(combined)
combined = featurize_columns(combined)
combined = featurize_date_columns(combined)
combined = encoding(combined)
combined = remove_constant_values(combined)
print('\ncombined shape after removing constant features->', combined.shape)
combined = remove_unwanted_features(combined)
combined = fill_missing(combined)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


combined shape after combining issues -> (13638, 306)
converting all text columns in lowercase.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  after removing the cwd from sys.path.


Removing constant columns -> 
application, issue.24, issue.25, issue.26, languageisocode, originatingbody_type, parties.2, respondent.3, respondent.4, documentcollectionid=CASELAW, documentcollectionid=JUDGMENTS, documentcollectionid=ENG, 
combined shape after removing constant features-> (13638, 294)


In [15]:
combined.to_csv('combined_inbetween.csv',index=False)

In [16]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score

In [17]:
target_col = 'importance'

In [18]:
combined_train = combined.query('label == "train"').drop(['issues', 'label'] , axis=1)

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(combined_train.drop([target_col], axis=1),combined_train[target_col],test_size=0.2,stratify=combined_train[target_col])

xgbm = LGBMClassifier(max_depth=6, learning_rate=0.1, n_estimators=500,
                         min_child_weight=100, subsample=1.0, 
                         colsample_bytree=0.8, colsample_bylevel=0.8,
                         random_state=42, n_jobs=-1)

oof_preds = cross_val_predict(xgbm, X_train, Y_train, cv=5, 
                                  n_jobs=-1, method="predict")
print("cv score: ", accuracy_score(oof_preds, Y_train) * 100)

In [20]:
XG = XGBClassifier()

In [21]:
oof_preds = cross_val_predict(XG, X_train, Y_train, cv=5, 
                                  n_jobs=-1, method="predict")
print("cv score: ", accuracy_score(oof_preds, Y_train) * 100)

cv score:  88.76372852717545


tst = combined.query('label == "test"').drop(['issues', 'label', target_col] , axis=1)
xgbm.fit(X_train,Y_train)
preds = xgbm.predict(tst)
preds 

In [22]:
tst = combined.query('label == "test"').drop(['issues', 'label', target_col] , axis=1)
XG.fit(X_train,Y_train)
preds = XG.predict(tst)
preds

array([4, 4, 4, ..., 4, 4, 4], dtype=int64)

In [23]:
sub = pd.DataFrame(columns=["appno","importance"])
sub["appno"] = test.appno
sub["importance"] = preds
sub.to_csv("result2.csv", index=False)